In [2]:
import numpy as np
import random
import math

In [23]:

def sigm(x):
    return 1/(1+math.exp(-x))

def sigm_delta(x):
    return sigm(x)*(1-sigm(x))

def softmax(nets):
    max_net = max(nets)
    exp_nets = [math.exp(net - max_net) for net in nets]
    sum_exp_nets = sum(exp_nets)
    softmax_values = [i / sum_exp_nets for i in exp_nets]
    return softmax_values

def oneHotEncoder(predictions_, dataset):
      output = []
      max_value = max(predictions_)
      index = predictions_.index(max_value)
      if dataset == 'digits':
        for i in range(len(predictions_)):
            if i == index:
              output.append(1.0)
            else:
              output.append(0.0)
      if dataset == 'xor':
        if predictions_[0]>=0.5:
            output = [0.0, 1.0]
        else:
            output = [1.0, 0.0]
      return output


In [3]:
class Layer:
    def __init__(self):
        #matrix of output weights for each neuron
        self.outputs = []

In [4]:
class InputLayer(Layer):
    def __init__(self, inputs=[]):
        super().__init__()

        #for first layer output weights are equal to the input
        self.outputs = inputs

        #first layer has # of neurons equal to the size of our input
        self.neurons_num = len(inputs)

        print(f"First layer added. It has {self.neurons_num} neurons")

In [5]:
class HiddenLayer(Layer):
    def __init__(self, neurons_num):
        super().__init__()
        self.nets = []
        self.deltas = []
        self.delta_w = []
        self.weights = []
        self.neurons_num = neurons_num

    def backpropagation(self, next_layer):
        for j in range(self.neurons_num):
            self.deltas[j] = sum(next_layer.weights[i][j+1]*next_layer.deltas[i] for i in range(next_layer.neurons_num)) * sigm_delta(self.nets[j])


    def propagateInputs(self, prev_layer, layers, dataset):
        for j in range(self.neurons_num):
            self.nets[j] = self.weights[j][0] + np.dot(self.weights[j][1:],prev_layer.outputs)
            if(prev_layer != layers[-2] or dataset == 'xor'):
              self.outputs[j] = sigm(self.nets[j])
        #for digits dataset we need softmax as activation function for output layer
        if(prev_layer == layers[-2] and dataset == 'digits'):
          self.outputs = softmax(self.nets).copy()



    def accumulateChange(self, prev_layer):
        for j in range(self.neurons_num):
            for i in range(1, prev_layer.neurons_num+1):
                self.delta_w[j][i] += self.deltas[j]*prev_layer.outputs[i-1]
            self.delta_w[j][0] = self.delta_w[j][0] + self.deltas[j]


    def adjustWeights(self, prev_layer, lr, u):
        for j in range(self.neurons_num):
            for i in range(1, prev_layer.neurons_num+1):
                self.weights[j][i] = self.weights[j][i] - lr*self.delta_w[j][i] - u*lr*self.delta_w[j][i]
            self.weights[j][0] -= (lr * self.delta_w[j][0] + u*lr*self.delta_w[j][0])



In [25]:

class NeuralNetwork:

    def __init__(self, inputs, real_outputs, layers_num, neurons_per_layer=[]):
        #matrix of patterns
        self.inputs = inputs
        #matrix of real outputs
        self.outputs = real_outputs
        self.layers = []
        #predicted output weights for current pattern
        self.output = []
        #real output for current pattern
        self.d = []
        self.layers_num = layers_num
        self.neurons_per_layer = neurons_per_layer
        print("Neurons per layer:" , self.neurons_per_layer[0])

        #initialization of network with first pattern
        self.layers.append(InputLayer(self.inputs[0]))
        print(f"Layer 0 output: {self.layers[0].outputs}")

        #adding hidden layers with desired # of neurons per layer
        for i in range(1, layers_num):
            self.add(HiddenLayer(neurons_per_layer[i]))
            print(f"Added Hidden: {neurons_per_layer[i]} neurons")


    def add(self, layer: Layer):
        self.layers.append(layer)

    def init_weights(self):
        #for each layer except the InputLayer
        for h in range(1, len(self.layers)):
        #for each neuron
            for i in range(self.layers[h].neurons_num):
                #number of weights is equal to number of neurons in previous layer + 1 for bias
                initial_weights = [random.uniform(-1, 1) for _ in range(self.layers[h-1].neurons_num+1)]
                self.layers[h].weights.append(initial_weights)
                self.layers[h].delta_w.append(np.zeros(self.layers[h-1].neurons_num+1))
                self.layers[h].deltas.append(np.zeros(1))
                self.layers[h].nets.append(np.zeros(1))
                self.layers[h].outputs.append(np.zeros(1))


    def backpropagateError(self):
        #network output are the output weights of last layer
        self.output = self.layers[-1].outputs
        for j in range(self.layers[-1].neurons_num):
          if self.dataset == 'xor':
            #derivatives for sigmoid function
            self.layers[-1].deltas[j] = -(self.d[j] - self.output[j]) * sigm_delta(self.layers[-1].nets[j])
          if self.dataset == 'digits':
            #derivatives for sotfmax function
            self.layers[-1].deltas[j] = 0
            for i in range(self.layers[-1].neurons_num):
              if i != j:
                self.layers[-1].deltas[j] -= (self.d[i] - self.output[i])*self.output[j]*(-self.output[i])
              else:
                self.layers[-1].deltas[j] -= (self.d[i] - self.output[i])*self.output[j]*(1 - self.output[i])

        for h in range(len(self.layers)-2, 0, -1):
            self.layers[h].backpropagation(self.layers[h+1])

    def forward(self):
        for h in range(1,len(self.layers)):
            self.layers[h].propagateInputs(self.layers[h-1], self.layers, self.dataset)


    def Change(self):
        for h in range(1, len(self.layers)):
            self.layers[h].accumulateChange(self.layers[h-1])

    def Weights(self, lr, u):
        for h in range(1, len(self.layers)):
            self.layers[h].adjustWeights(self.layers[h-1], lr, u)

    def feed_input(self, input, output):
        self.layers[0].outputs = input
        self.d = output
        for h in range(1, len(self.layers)):
            for i in range(self.layers[h].neurons_num):
                self.layers[h].delta_w[i] = np.zeros(len(self.layers[h-1].outputs)+1)

    def train(self, epochs, lr, u, dataset):
        self.dataset = dataset
        predicted_final = []
        for n in range(epochs):
            predictions = [[0.0] * len(self.outputs[0]) for _ in range(len(self.inputs))]
            index_list = [j for j in range(len(self.inputs))]
            while(index_list):
                i = random.choice(index_list)
                self.feed_input(self.inputs[i], self.outputs[i])
                self.forward()
                self.backpropagateError()
                self.Change()
                self.Weights(lr, u)
                outs = self.output.copy()
                outs = oneHotEncoder(outs, self.dataset)
                predictions[i] = outs.copy()
                index_list.remove(i)

            print(f"Accuracy for epoch {n}: {accuracy_score(self.outputs, predictions)}")
            if n == epochs-1:
              predicted_final = predictions.copy()
        print("Training successful")
        return predicted_final



    def predict(self, pattern):
        self.layers[0].outputs = pattern
        self.forward()
        output = self.layers[-1].outputs.copy()
        output = oneHotEncoder(output, self.dataset)
        return output






In [30]:
from csv import reader

def load_dataset(file_path):
    with open(file_path, 'r') as file:
        #read first line
        # k - inputs
        # J - outputs
        # N - patterns
        k, J, N = map(int, file.readline().split())

        inputs = []
        outputs = []

        # read N patterns
        for _ in range(N):
            line = file.readline().split()

            pattern_inputs = list(map(float, line[:k]))
            pattern_outputs = list(map(float, line[k:]))

            inputs.append(pattern_inputs)
            outputs.append(pattern_outputs)

    return inputs, outputs

In [31]:
in_digits, out_digits = load_dataset('train_digits.dat')


#overview of out dataset

print("Pattern lenght: ", len(in_digits[0]))
print("Number of patterns: ", len(out_digits))


Pattern lenght:  256
Number of patterns:  1274


In [36]:
from sklearn.metrics import accuracy_score

neurons_num =  [256, 128, 64, 32, 10]

model = NeuralNetwork(in_digits, out_digits, 5, neurons_num)
model.init_weights()



Neurons per layer: 256
First layer added. It has 256 neurons
Layer 0 output: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [37]:
predictions_digits = model.train(9, 0.8, 0.01, 'digits')

Accuracy for epoch 0: 0.3893249607535322
Accuracy for epoch 1: 0.6962323390894819
Accuracy for epoch 2: 0.8084772370486656
Accuracy for epoch 3: 0.8744113029827315
Accuracy for epoch 4: 0.8893249607535322
Accuracy for epoch 5: 0.9262166405023547
Accuracy for epoch 6: 0.9215070643642073
Accuracy for epoch 7: 0.9167974882260597
Accuracy for epoch 8: 0.9411302982731554
Training successful


In [38]:
# for test set
#optimal accuracy for 9 epochs

in_digits_test, out_digits_test = load_dataset('test_digits.dat')

pred_digits = []


for pattern in in_digits_test:
    output = model.predict(pattern)
    pred_digits.append(output.copy())



In [39]:
print("Accuracy for train set:", accuracy_score(out_digits, predictions_digits))
print("Accuracy for test set:", accuracy_score(out_digits_test, pred_digits))

#łiii

Accuracy for train set: 0.9411302982731554
Accuracy for test set: 0.890282131661442


In [26]:
in_xor, out_xor = load_dataset('train_xor.dat')

print(len(in_xor[0]))
print(len(out_xor))

neurons_num =  [2, 2, 1]

model_xor = NeuralNetwork(in_xor, out_xor, 3, neurons_num)
model_xor.init_weights()



2
4
Neurons per layer: 2
First layer added. It has 2 neurons
Layer 0 output: [1.0, -1.0]
Added Hidden: 2 neurons
Added Hidden: 1 neurons


In [27]:
predictions_xor = model_xor.train(10, 0.9  , 0.05, 'xor')

Accuracy for epoch 0: 0.5
Accuracy for epoch 1: 0.5
Accuracy for epoch 2: 0.5
Accuracy for epoch 3: 0.5
Accuracy for epoch 4: 0.5
Accuracy for epoch 5: 0.5
Accuracy for epoch 6: 0.75
Accuracy for epoch 7: 0.75
Accuracy for epoch 8: 0.5
Accuracy for epoch 9: 1.0
Training successful


In [28]:

print(accuracy_score(out_xor, predictions_xor))

1.0


In [29]:
# for test set

in_xor_test, out_xor_test = load_dataset('test_xor.dat')

pred_xor = []

for pattern in in_xor_test:
    output = model_xor.predict(pattern)
    print(output)
    pred_xor.append(output.copy())

print(f"Accuracy for train set: {accuracy_score(out_xor, predictions_xor)}")
print(f"Accuracy for test set: {accuracy_score(out_xor_test, pred_xor)}")

[1.0, 0.0]
[1.0, 0.0]
[1.0, 0.0]
[0.0, 1.0]
Accuracy for train set: 1.0
Accuracy for test set: 0.75
